In [1]:
# Download and extract the data.
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-07-12 13:23:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [   <=>              ]  41.00M  97.4MB/s    in 0.4s    

2023-07-12 13:23:56 (97.4 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, concatenate_datasets

data_files = {
    'train': 'drugsComTrain_raw.tsv',
    'test': 'drugsComTest_raw.tsv'
}
drug_dataset = load_dataset('csv', data_files=data_files, delimiter='\t')
drug_dataset = concatenate_datasets([drug_dataset['train'], drug_dataset['test']])
drug_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ec41c87d452ec392/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 215063
})

In [4]:
# Chaining the Dataset.shuffle() and Dataset.select() functions together.

drug_sample =drug_dataset.shuffle(seed=42).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [132558, 129380, 172605],
 'drugName': ['Doxepin', 'Levonorgestrel', 'Clonazepam'],
 'condition': ['Insomnia', 'Birth Control', 'Panic Disorde'],
 'review': ['"I couldn&#039;t stay asleep, tried Lunesta and Ambien, they had side effects I couldn&#039;t tolerate.  Sleep center said I had mild apnea and said their monitors showed me sleeping but I felt like I was awake more than they said I was and was literally exhausted every day, felt like I was on the verge of &quot;losing it,&quot; crying a lot.  Was not sad/depressed--was sleep deprived.  Have been taking 2 Silenor a night for 2 years:  sleep like a baby, 8 hours required, wake up rested, functioning beautifully--I have my life back."',
  '"I got Skyla 2 years ago (around $2000 total for 3 years) , and want to get another one the next summer (3 years limit). The insertion of course is never fun, no surgery of any form will be - but I was well enough to enjoy movies that night with friends. My periods have gotten ligh

In [5]:
# Verify that the number of IDs matches the number of rows in each split.

assert len(drug_dataset) == len(drug_dataset.unique('Unnamed: 0'))

In [6]:
# Rename 'Unnamed: 0' column to something interpretable.

drug_dataset = drug_dataset.rename_column(
    original_column_name='Unnamed: 0',
    new_column_name='patient_id'
)
drug_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 215063
})

In [7]:
# Number of unique drugs
print("Number of unique drugs: {}".format(len(drug_dataset.unique('drugName'))))

Number of unique drugs: 3671


In [8]:
# Number of unique conditions
print("Number of unique conditions: {}".format(len(drug_dataset.unique('condition'))))

Number of unique conditions: 917


In [9]:
# Drop rows using Dataset.filter() method
drug_dataset = drug_dataset.filter(lambda x: x['condition'] is not None)

Filter:   0%|          | 0/215063 [00:00<?, ? examples/s]

In [10]:
# Normalize
def lowercase_condition(inp):
    return {'condition': inp['condition'].lower()}

In [11]:
drug_dataset = drug_dataset.map(lowercase_condition)
drug_dataset['condition'][:3]

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

['left ventricular dysfunction', 'adhd', 'birth control']

In [12]:
drug_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 213869
})

#### **Creating new columns**

In [13]:
# Define a simple function that counts the number of words in each review.

def compute_review_length(inp):
    return {'review_length': len(inp['review'].split())}

# In this case, when `compute_review_length` function is passed to Dataset.map(), it will be applied to all the rows in the dataset to create
# a new `review_length` column.

drug_dataset = drug_dataset.map(compute_review_length)
drug_dataset[0]

Map:   0%|          | 0/213869 [00:00<?, ? examples/s]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [14]:
# Sort the datasets using Dataset.sort() to see what extreme values look like.
drug_dataset.sort('review_length')[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [15]:
# Alternate way to add new columns to the dataset. -> `Dataset.add_column()`

# Remove reviews that contain fewer than 30 words.

drug_dataset = drug_dataset.filter(lambda x: x['review_length'] > 30)
print(drug_dataset.num_rows)

Filter:   0%|          | 0/213869 [00:00<?, ? examples/s]

184622


In [16]:
import html
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x['review'])})

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

In [17]:
drug_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
    num_rows: 184622
})

In [18]:
# Count frequency of conditions
conditions = drug_dataset['condition']
from pprint import pprint

frequency = {}
for c in conditions:
    if c not in frequency.keys():
        frequency[c] = 1
    else:
        frequency[c] += 1

pprint(frequency)

{'0</span> users found this comment helpful.': 119,
 '100</span> users found this comment helpful.': 1,
 '105</span> users found this comment helpful.': 1,
 '10</span> users found this comment helpful.': 32,
 '11</span> users found this comment helpful.': 10,
 '121</span> users found this comment helpful.': 1,
 '123</span> users found this comment helpful.': 1,
 '12</span> users found this comment helpful.': 16,
 '135</span> users found this comment helpful.': 1,
 '13</span> users found this comment helpful.': 25,
 '142</span> users found this comment helpful.': 1,
 '145</span> users found this comment helpful.': 1,
 '146</span> users found this comment helpful.': 1,
 '14</span> users found this comment helpful.': 15,
 '15</span> users found this comment helpful.': 21,
 '16</span> users found this comment helpful.': 10,
 '17</span> users found this comment helpful.': 17,
 '18</span> users found this comment helpful.': 13,
 '19</span> users found this comment helpful.': 9,
 '1</span> us

In [19]:
# Add frequency of occurance of conditions as a column to the dataset.
def add_frequency(inp):
    return {'condition_frequency': frequency[inp['condition']]}

drug_dataset = drug_dataset.map(add_frequency)
drug_dataset

Map:   0%|          | 0/184622 [00:00<?, ? examples/s]

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
    num_rows: 184622
})

In [20]:
# Remove all conditions that occur less than 20 times.
drug_dataset = drug_dataset.filter(lambda x: x['condition_frequency'] >= 20)
drug_dataset.num_rows

Filter:   0%|          | 0/184622 [00:00<?, ? examples/s]

181875

In [21]:
drug_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
    num_rows: 181875
})

### **Creating a Validation set**

In [22]:
drug_dataset = drug_dataset.class_encode_column("condition", include_nulls=True)
drug_dataset

Flattening the indices:   0%|          | 0/181875 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/181875 [00:00<?, ? examples/s]

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
    num_rows: 181875
})

In [23]:
# Datasets provides a Dataset.train_test_split() function that is based on the famous functionality from scikit-learn.
drug_dataset_clean = drug_dataset.train_test_split(train_size=0.9, seed=42, stratify_by_column='condition')
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 163687
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 18188
    })
})

In [24]:
test_dataset = drug_dataset_clean.pop('test')
test_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
    num_rows: 18188
})

In [28]:
# Create validation set from train set.
drug_dataset_clean = drug_dataset_clean['train'].train_test_split(train_size=0.9, seed=42, stratify_by_column='condition')
drug_dataset_clean['validation'] = drug_dataset_clean.pop('test') # Rename the newly created test set to validation set.
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 16369
    })
})

In [29]:
# Add the "test" set to our DatasetDict
drug_dataset_clean["test"] = test_dataset
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 147318
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 16369
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'condition_frequency'],
        num_rows: 18188
    })
})

### **Saving the datasets**

In [30]:
# For CSV and JSON formats, we have to store each split as a separate file.
for split, dataset in drug_dataset_clean.items():
    dataset.to_csv(f"drug-reviews-{split}.csv")

Creating CSV from Arrow format:   0%|          | 0/148 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/19 [00:00<?, ?ba/s]